In [ ]:
import pandas as pd
import scipy.sparse as sps
from urllib.request import urlretrieve
import zipfile, os
# If file exists, skip the download
data_file_path = "Data_manager_split_datasets/Movielens10M/"
data_file_name = data_file_path + "movielens_10m.zip"

# If directory does not exist, create
if not os.path.exists(data_file_path):
    os.makedirs(data_file_path)

if not os.path.exists(data_file_name):
    urlretrieve ("http://files.grouplens.org/datasets/movielens/ml-10m.zip", data_file_name)
dataFile = zipfile.ZipFile(data_file_path + "movielens_10m.zip")

URM_path = dataFile.extract("ml-10M100K/ratings.dat", path = data_file_path + "decompressed/")

URM_file = open(URM_path, 'r')
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep="::", 
                                header=None, 
                                dtype={0:int, 1:int, 2:float, 3:int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction", "Timestamp"]
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)
URM_all_dataframe["UserID"] = URM_all_dataframe["UserID"].map(user_original_ID_to_index)
URM_all_dataframe["ItemID"] = URM_all_dataframe["ItemID"].map(item_original_ID_to_index)
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))

URM_all.tocsr()
train_test_split = 0.80

n_interactions = URM_all.nnz




In [6]:
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender

ImportError: cannot import name 'TwoRandomWalksSimilarity' from 'Diffusion.similarity_models' (/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Diffusion/similarity_models.py)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens100KReader import Movielens100KReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens100KReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data() #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

In [ ]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens1MReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/data') #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

In [ ]:
import optuna
import numpy as np
import pandas as pd
import os
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender

def objective(trial):

    cutoff = 10
    metric = 'NDCG'
    directory_path = '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Self-Attention/OptunaResults/Movielens100K'

    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256, 512, 1024])
    embeddings_dim = trial.suggest_categorical('embeddings_dim', [64, 128, 256, 512,1024])
    heads = trial.suggest_categorical('heads', [1])
    attention_blocks = trial.suggest_categorical('attention_blocks', [2, 3, 5, 8, 13])
    d_ff = trial.suggest_categorical('d_ff', [512, 1024, 2048])
    epochs = trial.suggest_int('epochs', 200, 600)
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    noise_timesteps = trial.suggest_int('noise_timesteps', 1, 2000)
    inference_timesteps = trial.suggest_int('inference_timesteps', 1, noise_timesteps-1)
    start_beta = trial.suggest_float('start_beta', 0.00001, 0.001)
    end_beta = trial.suggest_float('end_beta', 0.01, 0.2)

    # Initialize and train the recommender

    diffusion_model = MultiBlockAttentionDiffusionRecommender(URM_train = URM_train, verbose = False, use_gpu = True)

    diffusion_model.fit(
                      epochs=epochs,
                      batch_size=batch_size,
                      embeddings_dim=embeddings_dim,
                      heads=heads,
                      attention_blocks = attention_blocks,
                      d_ff = d_ff,
                      l2_reg=l2_reg,
                      learning_rate=learning_rate,
                      noise_timesteps = noise_timesteps,
                      inference_timesteps = inference_timesteps,
                      start_beta = start_beta,
                      end_beta = end_beta
    )

    result_df, _ = evaluator_validation.evaluateRecommender(diffusion_model)
    hyperparams = {
    'batch_size': batch_size,
    'embeddings_dim': embeddings_dim,
    'heads': heads,
    'attention_blocks': attention_blocks,
    'd_ff': d_ff,
    'epochs': epochs,
    'l2_reg': l2_reg,
    'learning_rate': learning_rate,
    'noise_timesteps': noise_timesteps,
    'inference_timesteps': inference_timesteps,
    'start_beta': start_beta,
    'end_beta': end_beta}

    result_df['hyperparams'] = str(hyperparams)

    filename = directory_path + diffusion_model.RECOMMENDER_NAME + ".csv"

    # Check if file exists
    if os.path.isfile(filename):
        # If it exists, append without writing the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
    else:
        # If it doesn't exist, create it, write the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)

    return result_df.loc[cutoff][metric]



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50,show_progress_bar=True)

In [ ]:
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender
diffusion_model = MultiBlockAttentionDiffusionRecommender(URM_train = URM_train, use_gpu = True)
diffusion_model.fit(epochs=5)

In [ ]:
result_df, _ = evaluator_validation.evaluateRecommender(diffusion_model)
result_df